# Generate training data

### CISCO and Spaninsh Ens

In [1]:
import pandas as pd
import re

# ==============================================================================
# 1. CONFIGURAZIONE
# ==============================================================================
CISCO_PATH = "Schemes/Cisco.csv"         # Assicurati che il percorso sia corretto
ENS_PATH = "Schemes/SpanishENS.csv"      # Assicurati che il percorso sia corretto
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step1_cisco_ens.csv"

# ==============================================================================
# 2. CARICAMENTO DATI
# ==============================================================================
print("1. Caricamento file...")

# Caricamento Cisco
df_cisco = pd.read_csv(CISCO_PATH)
print(f"   - Cisco loaded: {df_cisco.shape} righe")

# Caricamento Spanish ENS (con separatore punto e virgola)
# Usiamo dtype=str per evitare problemi con codici interpretati come numeri
df_ens = pd.read_csv(ENS_PATH, sep=';', dtype=str)
print(f"   - Spanish ENS loaded: {df_ens.shape} righe")

# ==============================================================================
# 3. CREAZIONE LOOKUP TABLE (ENS INGLESE AGGREGATO)
# ==============================================================================
print("\n2. Indicizzazione e Aggregazione ENS (English)...")

ens_lookup = {}

# Raggruppiamo per codice perché nel CSV lo stesso codice può apparire più volte
# (es. una riga per il titolo, una riga per le domande)
grouped_ens = df_ens.groupby('Codice')

for code, group in grouped_ens:
    # Normalizza la chiave (codice)
    if pd.isna(code): continue
    clean_code = str(code).strip().lower()
    
    # Raccogli tutti i testi INGLESI disponibili per questo codice
    texts = []
    
    for _, row in group.iterrows():
        # Titolo Inglese
        if 'Titolo_EN' in row and not pd.isna(row['Titolo_EN']):
            t = str(row['Titolo_EN']).strip()
            if t: texts.append(t)
            
        # Domande Inglese
        if 'Domande_EN' in row and not pd.isna(row['Domande_EN']):
            q = str(row['Domande_EN']).strip()
            # Rimuovi i pipe '|' che separano le domande e sostituisci con spazio
            q = q.replace('|', ' ')
            if q: texts.append(q)
            
    # Unisci tutto il testo raccolto per questo controllo
    full_text_en = ". ".join(texts)
    
    # Pulizia finale (spazi doppi, ecc)
    full_text_en = re.sub(r'\s+', ' ', full_text_en).strip()
    
    if full_text_en:
        ens_lookup[clean_code] = full_text_en

print(f"   - Indicizzati {len(ens_lookup)} codici ENS univoci con testo in Inglese.")

# ==============================================================================
# 4. MAPPING E GENERAZIONE DATASET
# ==============================================================================
print("\n3. Generazione coppie di training (Cisco -> ENS)...")

training_data = []
stats = {
    "matches": 0,
    "missing": set()
}

# Colonne Cisco dove cercare i riferimenti ENS
ens_columns = ['Spanish ENS BASIC Control', 'Spanish ENS Medium Control', 'Spanish ENS High Control']

for idx, row in df_cisco.iterrows():
    # Anchor (Cisco)
    # Combiniamo Titolo e Wording per la massima ricchezza semantica
    if pd.isna(row['Control Reference']): continue
    
    anchor_id = str(row['Control Reference'])
    title = str(row['Control Title']) if not pd.isna(row['Control Title']) else ""
    wording = str(row['Control Wording']) if not pd.isna(row['Control Wording']) else ""
    
    anchor_text = f"{title}: {wording}".strip()
    
    # Target (ENS)
    # Cerchiamo i codici nelle 3 colonne
    target_codes = []
    for col in ens_columns:
        if col in row and not pd.isna(row[col]):
            # Split per virgola in caso ci siano più controlli in una cella
            codes_in_cell = str(row[col]).split(',')
            target_codes.extend([c.strip().lower() for c in codes_in_cell])
    
    # Rimuovi duplicati e itera
    unique_targets = set(target_codes)
    
    for t_code in unique_targets:
        if not t_code: continue
        
        if t_code in ens_lookup:
            # MATCH TROVATO
            positive_text = ens_lookup[t_code]
            
            training_data.append({
                'anchor_id': anchor_id,
                'anchor_text': anchor_text,
                'target_id': t_code,
                'target_text': positive_text,
                'source': 'Cisco',
                'target': 'Spanish ENS (EN)'
            })
            stats["matches"] += 1
        else:
            stats["missing"].add(t_code)

# ==============================================================================
# 5. SALVATAGGIO
# ==============================================================================
df_train = pd.DataFrame(training_data)
df_train.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO FINALE")
print("-" * 30)
print(f"Totale coppie generate: {len(df_train)}")
print(f"Codici mancanti (nel file ENS): {len(stats['missing'])}")
if stats['missing']:
    print(f"Esempio mancanti: {list(stats['missing'])[:5]}")
print(f"\nFile salvato come: {OUTPUT_FILE}")

# Anteprima
print("\nAnteprima dati:")
print(df_train[['anchor_text', 'target_text']].head(2))

1. Caricamento file...
   - Cisco loaded: (713, 32) righe
   - Spanish ENS loaded: (213, 8) righe

2. Indicizzazione e Aggregazione ENS (English)...
   - Indicizzati 169 codici ENS univoci con testo in Inglese.

3. Generazione coppie di training (Cisco -> ENS)...
------------------------------
RISULTATO FINALE
------------------------------
Totale coppie generate: 318
Codici mancanti (nel file ENS): 28
Esempio mancanti: ['mp.info.2', 'mp.com.9', 'mp.s.1', 'mp.eq.2', 'mp.info.9']

File salvato come: TrainAndTestData/trainingData/training_data_step1_cisco_ens.csv

Anteprima dati:
                                         anchor_text  \
0  Control Self-Assessments: Independent Control ...   
1  Control Self-Assessments: Independent Control ...   

                                         target_text  
0  Metrics system. Taking into account the securi...  
1  Certified components. Is the Catalog of Inform...  


### CISCO and Spaninsh Ens

In [2]:
import pandas as pd
import os

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
# Percorsi basati sulla tua struttura di cartelle
PATH_CISCO = "Schemes/Cisco.csv"
PATH_SECNUM = "Schemes/Secnumcloud.csv"
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step2_cisco_secNumCloud.csv"

# ==============================================================================
# 2. CARICAMENTO E INDICIZZAZIONE SECNUMCLOUD
# ==============================================================================
print("1. Caricamento e indicizzazione SecNumCloud...")

try:
    # Tentativo di lettura standard (virgola)
    df_sec = pd.read_csv(PATH_SECNUM)
    
    # Verifica colonne critiche
    if 'ID' not in df_sec.columns or 'Description_EN' not in df_sec.columns:
        # Fallback: prova con punto e virgola se la prima lettura non ha parsato le colonne
        df_sec = pd.read_csv(PATH_SECNUM, sep=';')

    print(f"   - SecNumCloud caricato: {len(df_sec)} righe.")
except Exception as e:
    print(f"   - ERRORE critico nel caricamento di SecNumCloud: {e}")
    exit()

# Creazione dizionario di lookup: ID (lowercase) -> Descrizione Inglese
secnum_lookup = {}
for _, row in df_sec.iterrows():
    if pd.isna(row['ID']): continue
    
    # Normalizzazione ID (es: "5.1.A " -> "5.1.a")
    clean_id = str(row['ID']).strip().lower()
    
    # Estrazione testo inglese
    if 'Description_EN' in row and not pd.isna(row['Description_EN']):
        desc = str(row['Description_EN']).strip()
        secnum_lookup[clean_id] = desc

print(f"   - Indicizzati {len(secnum_lookup)} controlli SecNumCloud univoci (EN).")

# ==============================================================================
# 3. MAPPING CISCO -> SECNUMCLOUD
# ==============================================================================
print("\n2. Generazione coppie di training (Cisco -> SecNumCloud)...")

df_cisco = pd.read_csv(PATH_CISCO)
print(f"   - Cisco caricato: {len(df_cisco)} righe.")

training_data = []
stats = {
    "matches": 0,
    "missing": set()
}

# La colonna Cisco che contiene i riferimenti
target_col = 'SecNumCloud Control'

for idx, row in df_cisco.iterrows():
    # Verifica se c'è un mapping
    if target_col not in row or pd.isna(row[target_col]):
        continue
    
    # 1. Preparazione Anchor (Cisco)
    anchor_id = str(row['Control Reference'])
    title = str(row['Control Title']) if not pd.isna(row['Control Title']) else ""
    wording = str(row['Control Wording']) if not pd.isna(row['Control Wording']) else ""
    
    # Uniamo titolo e wording per dare contesto completo al modello
    anchor_text = f"{title}: {wording}".strip()
    
    # 2. Preparazione Target (SecNumCloud IDs)
    # Cisco può contenere liste tipo "5.1.a, 5.2.b" o usare newline
    raw_refs = str(row[target_col]).replace('\n', ',')
    target_ids_list = [t.strip().lower() for t in raw_refs.split(',')]
    
    # 3. Matching
    for t_id in set(target_ids_list): # set() per evitare duplicati sulla stessa riga
        if not t_id: continue
        
        if t_id in secnum_lookup:
            # MATCH TROVATO
            positive_text = secnum_lookup[t_id]
            
            training_data.append({
                'anchor_id': anchor_id,
                'anchor_text': anchor_text,
                'target_id': t_id,      # ID normalizzato
                'target_text': positive_text,
                'source': 'Cisco',
                'target': 'SecNumCloud (EN)'
            })
            stats["matches"] += 1
        else:
            # MATCH NON TROVATO (ID presente in Cisco ma non nel file SecNumCloud)
            stats["missing"].add(t_id)

# ==============================================================================
# 4. SALVATAGGIO FILE STEP 2
# ==============================================================================
df_result = pd.DataFrame(training_data)
df_result.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO STEP 2")
print("-" * 30)
print(f"Nuove coppie generate: {len(df_result)}")
print(f"Codici SecNumCloud non trovati: {len(stats['missing'])}")
if stats['missing']:
    print(f"Esempio mancanti: {list(stats['missing'])[:3]}...")
print(f"\nFile salvato: {OUTPUT_FILE}")

# Anteprima
print("\nAnteprima dati:")
print(df_result[['anchor_id', 'target_id', 'target_text']].head(2))

1. Caricamento e indicizzazione SecNumCloud...
   - SecNumCloud caricato: 287 righe.
   - Indicizzati 261 controlli SecNumCloud univoci (EN).

2. Generazione coppie di training (Cisco -> SecNumCloud)...
   - Cisco caricato: 713 righe.
------------------------------
RISULTATO STEP 2
------------------------------
Nuove coppie generate: 567
Codici SecNumCloud non trovati: 33
Esempio mancanti: ['18.2.1.b', '9.6.k', '9.6.g']...

File salvato: TrainAndTestData/trainingData/training_data_step2_cisco_secNumCloud.csv

Anteprima dati:
  anchor_id target_id                                        target_text
0     CCF 1  18.2.1.a  The service provider must document and impleme...
1     CCF 1    18.4.a  The service provider must document and impleme...


### Cisco and BSIC5

In [3]:
import pandas as pd
import json
import os
import re

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
PATH_CISCO = "Schemes/Cisco.csv"
PATH_BSI = "Schemes/BSI-C5.json"
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step3_cisco_bsi.csv"

# ==============================================================================
# 2. CARICAMENTO E INDICIZZAZIONE BSI C5 (JSON)
# ==============================================================================
print("1. Caricamento e indicizzazione BSI C5...")

try:
    with open(PATH_BSI, 'r', encoding='utf-8') as f:
        bsi_data = json.load(f)
    print(f"   - BSI C5 JSON caricato. Totale elementi: {len(bsi_data)}")
except Exception as e:
    print(f"   - ERRORE CRITICO nel caricamento del JSON BSI: {e}")
    # In un notebook, fermiamo l'esecuzione qui se il file non c'è
    raise e

bsi_lookup = {}

for item in bsi_data:
    # Verifica che il codice esista
    if 'code' not in item or not item['code']:
        continue
        
    # Normalizzazione ID (es. "IAM-02" -> "iam-02")
    code = str(item['code']).strip().lower()
    
    # Costruzione del Testo Ricco
    # Combiniamo Nome, Titolo Descrizione e Descrizione Estesa
    name = item.get('name', '')
    desc_title = item.get('descriptionTitle', '')
    desc_ext = item.get('descriptionExtended', '')
    
    # Pulizia valori None/Null (che nel JSON appaiono come null)
    if name is None: name = ""
    if desc_title is None: desc_title = ""
    if desc_ext is None: desc_ext = ""
    
    # Unione stringhe
    full_text = f"{name}: {desc_title} {desc_ext}".strip()
    
    # Pulizia spazi extra e newline
    full_text = re.sub(r'\s+', ' ', full_text).strip()
    
    if full_text:
        bsi_lookup[code] = full_text

print(f"   - Indicizzati {len(bsi_lookup)} controlli BSI con testo.")

# ==============================================================================
# 3. MAPPING CISCO -> BSI C5
# ==============================================================================
print("\n2. Generazione coppie di training (Cisco -> BSI C5)...")

try:
    df_cisco = pd.read_csv(PATH_CISCO)
    print(f"   - Cisco CSV caricato: {len(df_cisco)} righe.")
except Exception as e:
    print(f"   - ERRORE CRITICO nel caricamento Cisco: {e}")
    raise e

training_data = []
stats = {
    "matches": 0,
    "missing": set()
}

# La colonna in Cisco si chiama esattamente "BSI C5"
target_col = 'BSI C5'

for idx, row in df_cisco.iterrows():
    # Verifica se c'è un mapping in questa riga
    if target_col not in row or pd.isna(row[target_col]):
        continue
        
    # 1. Preparazione Anchor (Cisco)
    anchor_id = str(row['Control Reference'])
    title = str(row['Control Title']) if not pd.isna(row['Control Title']) else ""
    wording = str(row['Control Wording']) if not pd.isna(row['Control Wording']) else ""
    
    anchor_text = f"{title}: {wording}".strip()
    anchor_text = re.sub(r'\s+', ' ', anchor_text) # pulizia veloce spazi
    
    # 2. Preparazione Target (BSI IDs)
    # I codici in Cisco sono separati da virgola (es. "OIS-01, SP-01")
    raw_refs = str(row[target_col]).replace('\n', ',')
    target_ids_list = [t.strip().lower() for t in raw_refs.split(',')]
    
    # 3. Matching
    for t_id in set(target_ids_list): # set() per rimuovere duplicati
        if not t_id: continue
        
        if t_id in bsi_lookup:
            # MATCH TROVATO
            positive_text = bsi_lookup[t_id]
            
            training_data.append({
                'anchor_id': anchor_id,
                'anchor_text': anchor_text,
                'target_id': t_id,      # ID normalizzato
                'target_text': positive_text,
                'source': 'Cisco',
                'target': 'BSI C5'
            })
            stats["matches"] += 1
        else:
            # MATCH NON TROVATO
            stats["missing"].add(t_id)

# ==============================================================================
# 4. SALVATAGGIO FILE STEP 3
# ==============================================================================
df_result = pd.DataFrame(training_data)
df_result.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO STEP 3")
print("-" * 30)
print(f"Nuove coppie generate: {len(df_result)}")
print(f"Codici BSI citati in Cisco ma non trovati nel JSON: {len(stats['missing'])}")
if stats['missing']:
    print(f"Esempi codici mancanti: {list(stats['missing'])[:5]}...")
print(f"\nFile salvato in: {OUTPUT_FILE}")

# Anteprima
print("\nAnteprima dati:")
print(df_result[['anchor_id', 'target_id', 'target_text']].head(2))

1. Caricamento e indicizzazione BSI C5...
   - BSI C5 JSON caricato. Totale elementi: 294
   - Indicizzati 223 controlli BSI con testo.

2. Generazione coppie di training (Cisco -> BSI C5)...
   - Cisco CSV caricato: 713 righe.
------------------------------
RISULTATO STEP 3
------------------------------
Nuove coppie generate: 555
Codici BSI citati in Cisco ma non trovati nel JSON: 0

File salvato in: TrainAndTestData/trainingData/training_data_step3_cisco_bsi.csv

Anteprima dati:
  anchor_id target_id                                        target_text
0     CCF 1    ois-01  Information Security Management System (ISMS):...
1     CCF 1     sp-02  Review and Approval of Policies and Instructio...


### Cisco and NewEUCS Requirements

In [4]:
import pandas as pd
import re
import os

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
PATH_CISCO = "Schemes/Cisco.csv"
PATH_EUCS = "Schemes/NewEucsRequirements_with_texts.csv"
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step4_cisco_eucs.csv"

# ==============================================================================
# 2. CARICAMENTO E INDICIZZAZIONE EUCS
# ==============================================================================
print("1. Caricamento e indicizzazione EUCS...")

try:
    # Tentativo primario con virgola (basato sul tuo 'head')
    df_eucs = pd.read_csv(PATH_EUCS, sep=',', encoding='utf-8')
    
    # Verifica se le colonne chiave esistono, altrimenti riprova con ;
    if 'EUCS Ref (Detailed)' not in df_eucs.columns:
        print("   - Separatore virgola fallito, provo punto e virgola...")
        df_eucs = pd.read_csv(PATH_EUCS, sep=';', encoding='utf-8')
        
    print(f"   - EUCS caricato: {len(df_eucs)} righe.")
except Exception as e:
    print(f"   - ERRORE CRITICO caricamento EUCS: {e}")
    # Fallback encoding latin-1 se utf-8 fallisce
    try:
        df_eucs = pd.read_csv(PATH_EUCS, sep=',', encoding='latin-1')
        print("   - EUCS caricato con encoding latin-1.")
    except:
        raise e

eucs_lookup = {}

for _, row in df_eucs.iterrows():
    # La chiave è il riferimento dettagliato (es. OIS-01.1)
    if 'EUCS Ref (Detailed)' not in row or pd.isna(row['EUCS Ref (Detailed)']):
        continue
        
    # Normalizzazione ID
    code = str(row['EUCS Ref (Detailed)']).strip().lower()
    
    # Estrazione testo
    text = ""
    if 'EUCS Text' in row and not pd.isna(row['EUCS Text']):
        text = str(row['EUCS Text']).strip()
        # Pulizia caratteri strani (es. newline o pipe)
        text = text.replace('\n', ' ').replace('|', ' ')
        text = re.sub(r'\s+', ' ', text)
        
    if code and text:
        eucs_lookup[code] = text

print(f"   - Indicizzati {len(eucs_lookup)} requisiti EUCS con testo.")

# ==============================================================================
# 3. MAPPING CISCO -> EUCS
# ==============================================================================
print("\n2. Generazione coppie di training (Cisco -> EUCS)...")

df_cisco = pd.read_csv(PATH_CISCO)
training_data = []
stats = {
    "matches": 0,
    "missing": set()
}

# Colonne target in Cisco (ne abbiamo 3 per l'EUCS)
target_columns = [
    'EUCS Basic Control', 
    'EUCS Substantial Control', 
    'EUCS High Control'
]

for idx, row in df_cisco.iterrows():
    # 1. Anchor (Cisco)
    anchor_id = str(row['Control Reference'])
    title = str(row['Control Title']) if not pd.isna(row['Control Title']) else ""
    wording = str(row['Control Wording']) if not pd.isna(row['Control Wording']) else ""
    anchor_text = f"{title}: {wording}".strip()
    anchor_text = re.sub(r'\s+', ' ', anchor_text)
    
    # 2. Raccolta Target IDs da tutte e 3 le colonne
    target_ids_list = []
    for col in target_columns:
        if col in row and not pd.isna(row[col]):
            # Split per virgola o newline
            raw_val = str(row[col]).replace('\n', ',')
            codes = [c.strip().lower() for c in raw_val.split(',')]
            target_ids_list.extend(codes)
            
    # 3. Matching
    for t_id in set(target_ids_list): # set() rimuove duplicati tra le colonne
        if not t_id: continue
        
        # Correzione typo comune (visto nel file Cisco: OSI -> OIS)
        if t_id.startswith('osi-'):
            t_id = t_id.replace('osi-', 'ois-')

        if t_id in eucs_lookup:
            # MATCH TROVATO
            positive_text = eucs_lookup[t_id]
            
            training_data.append({
                'anchor_id': anchor_id,
                'anchor_text': anchor_text,
                'target_id': t_id,
                'target_text': positive_text,
                'source': 'Cisco',
                'target': 'EUCS'
            })
            stats["matches"] += 1
        else:
            stats["missing"].add(t_id)

# ==============================================================================
# 4. SALVATAGGIO FILE STEP 4
# ==============================================================================
df_result = pd.DataFrame(training_data)
df_result.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO STEP 4")
print("-" * 30)
print(f"Nuove coppie generate: {len(df_result)}")
print(f"Codici EUCS non trovati: {len(stats['missing'])}")
if stats['missing']:
    print(f"Esempio mancanti: {list(stats['missing'])[:5]}...")
print(f"\nFile salvato in: {OUTPUT_FILE}")

# Anteprima
print("\nAnteprima dati:")
print(df_result[['anchor_id', 'target_id', 'target_text']].head(2))

1. Caricamento e indicizzazione EUCS...
   - EUCS caricato: 529 righe.
   - Indicizzati 522 requisiti EUCS con testo.

2. Generazione coppie di training (Cisco -> EUCS)...
------------------------------
RISULTATO STEP 4
------------------------------
Nuove coppie generate: 1272
Codici EUCS non trovati: 14
Esempio mancanti: ['rm-02.1b', 'cs-09.2', 'ois-01.4', 'ps-04.4a', 'cs-03.2a']...

File salvato in: TrainAndTestData/trainingData/training_data_step4_cisco_eucs.csv

Anteprima dati:
  anchor_id target_id                                        target_text
0     CCF 1   co-02.2  The CSP shall document and implement an audit ...
1     CCF 1   co-01.2  The CSP shall document and implement procedure...


### NewEucs and Fabasoft

In [7]:
import pandas as pd
import re
import os

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
PATH_EUCS = "Schemes/NewEucsRequirements_with_texts.csv"
PATH_FABASOFT = "Schemes/fabasoftMetrics.csv"
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step5_fabasoft_eucs.csv"

# ==============================================================================
# 2. CARICAMENTO E INDICIZZAZIONE EUCS (VIA BSI C5)
# ==============================================================================
print("1. Caricamento EUCS e indicizzazione via BSI C5...")

df_eucs = pd.DataFrame()

# Lettura file con pulizia colonne
try:
    # Usiamo engine python per gestire CSV complessi
    df_eucs = pd.read_csv(PATH_EUCS, sep=',', encoding='utf-8', engine='python', on_bad_lines='skip')
    
    # --- FIX CRUCIALE: RIMUOVIAMO SPAZI DAI NOMI DELLE COLONNE ---
    df_eucs.columns = df_eucs.columns.str.strip()
    
    print(f"   - EUCS caricato: {len(df_eucs)} righe.")
    print(f"   - Colonne trovate: {df_eucs.columns.tolist()}") # Debug
except Exception as e:
    print(f"   - Errore caricamento EUCS (utf-8): {e}")
    # Fallback latin-1 e separatore ;
    try:
        df_eucs = pd.read_csv(PATH_EUCS, sep=';', encoding='latin-1', engine='python', on_bad_lines='skip')
        df_eucs.columns = df_eucs.columns.str.strip()
        print(f"   - EUCS caricato (latin-1, ;): {len(df_eucs)} righe.")
    except Exception as e2:
        print(f"   - ERRORE CRITICO FILE EUCS: {e2}")

# Individuazione colonne target dopo lo strip
col_bsi = 'C5.2020 GERMANY'
col_eucs_id = 'EUCS Ref (Detailed)'
col_eucs_text = 'EUCS Text'

# Verifica esistenza colonna BSI
if col_bsi not in df_eucs.columns:
    print(f"   - ATTENZIONE: Colonna '{col_bsi}' non trovata anche dopo la pulizia!")
    # Tentativo di trovarla parzialmente
    candidates = [c for c in df_eucs.columns if "C5" in c and "GERMANY" in c]
    if candidates:
        col_bsi = candidates[0]
        print(f"   - Usando colonna alternativa: '{col_bsi}'")

bsi_to_eucs = {}
count_mapped = 0

if not df_eucs.empty and col_bsi in df_eucs.columns:
    for _, row in df_eucs.iterrows():
        # Estrazione dati EUCS
        e_id = str(row[col_eucs_id]).strip() if col_eucs_id in row and not pd.isna(row[col_eucs_id]) else ""
        e_text = str(row[col_eucs_text]).strip() if col_eucs_text in row and not pd.isna(row[col_eucs_text]) else ""
        
        if not e_id or not e_text: 
            continue
            
        # Estrazione dati BSI (Link)
        bsi_refs = str(row[col_bsi]) if not pd.isna(row[col_bsi]) else ""
        if not bsi_refs: 
            continue
            
        # Split per newline o virgola (es. "OIS-01\nOIS-03")
        refs = re.split(r'[\n,]', bsi_refs)
        
        for r in refs:
            clean_bsi = r.strip().lower()
            if clean_bsi:
                if clean_bsi not in bsi_to_eucs:
                    bsi_to_eucs[clean_bsi] = []
                bsi_to_eucs[clean_bsi].append((e_id, e_text))
                count_mapped += 1

print(f"   - Indicizzati {len(bsi_to_eucs)} codici BSI che puntano a {count_mapped} requisiti EUCS.")

# ==============================================================================
# 3. ELABORAZIONE FABASOFT METRICS
# ==============================================================================
print("\n2. Elaborazione Fabasoft Metrics -> EUCS (via BSI)...")

try:
    df_fab = pd.read_csv(PATH_FABASOFT, engine='python', on_bad_lines='skip')
    print(f"   - Fabasoft caricato: {len(df_fab)} righe.")
except Exception as e:
    print(f"   - ERRORE Fabasoft: {e}")
    df_fab = pd.DataFrame()

# Trova colonna riferimenti (contiene "Possible Control ID")
ref_col = None
for col in df_fab.columns:
    if "Possible Control ID" in col:
        ref_col = col
        break
if not ref_col and not df_fab.empty:
    ref_col = df_fab.columns[-1] # Fallback

print(f"   - Colonna Riferimenti Fabasoft: '{ref_col}'")

training_data = []
stats = {"matches": 0, "missing_bsi_link": set()}

if not df_fab.empty and ref_col and bsi_to_eucs:
    for idx, row in df_fab.iterrows():
        # Anchor (Metric)
        m_id = str(row.get('ID', ''))
        m_name = str(row.get('Name', ''))
        m_desc = str(row.get('Description', ''))
        
        # Pulizia nan
        if m_id.lower() == 'nan': m_id = ""
        if m_name.lower() == 'nan': m_name = ""
        if m_desc.lower() == 'nan': m_desc = ""
        
        anchor_parts = [p for p in [m_id, m_name, m_desc] if p]
        anchor_text = ": ".join(anchor_parts)
        anchor_text = re.sub(r'\s+', ' ', anchor_text).strip()
        
        if not anchor_text: continue
        
        # Estrazione Referenze BSI da Fabasoft
        raw_refs = str(row[ref_col]) if not pd.isna(row[ref_col]) else ""
        lines = raw_refs.split('\n')
        
        for line in lines:
            line = line.strip()
            target_bsi = None
            
            # Cerca "BSI C5 OPS-22"
            if "BSI C5" in line:
                target_bsi = re.sub(r'BSI\s+C5\s+', '', line, flags=re.IGNORECASE).strip()
            
            if target_bsi:
                clean_bsi = target_bsi.lower()
                
                # BRIDGE: Fabasoft(BSI) -> EUCS(BSI)
                if clean_bsi in bsi_to_eucs:
                    eucs_targets = bsi_to_eucs[clean_bsi]
                    for (e_id, e_text) in eucs_targets:
                        training_data.append({
                            'anchor_id': m_id,
                            'anchor_text': anchor_text,
                            'target_id': e_id,
                            'target_text': e_text,
                            'source': 'Fabasoft Metrics',
                            'target': 'EUCS (via BSI C5)'
                        })
                        stats["matches"] += 1
                else:
                    stats["missing_bsi_link"].add(clean_bsi)

# ==============================================================================
# 4. SALVATAGGIO
# ==============================================================================
df_result = pd.DataFrame(training_data, columns=['anchor_id', 'anchor_text', 'target_id', 'target_text', 'source', 'target'])
df_result.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO STEP 5")
print("-" * 30)
print(f"Nuove coppie generate: {len(df_result)}")
if stats['missing_bsi_link']:
    print(f"Codici BSI in Fabasoft non mappati in EUCS: {len(stats['missing_bsi_link'])}")
print(f"\nFile salvato in: {OUTPUT_FILE}")

if not df_result.empty:
    print(df_result[['anchor_id', 'target_id']].head(3))

1. Caricamento EUCS e indicizzazione via BSI C5...
   - EUCS caricato: 529 righe.
   - Colonne trovate: ['EUCS Category', 'EUCS Control (2022)', 'Control_Code', 'EUCS Ref (Detailed)', 'EUCS Text', 'EUCS Ass. Level', 'Code', 'C5.2020 GERMANY', 'SecNumCloud FRANCE', 'ISO 27002', 'ISO 27017']
   - Indicizzati 110 codici BSI che puntano a 576 requisiti EUCS.

2. Elaborazione Fabasoft Metrics -> EUCS (via BSI)...
   - Fabasoft caricato: 57 righe.
   - Colonna Riferimenti Fabasoft: 'Possible Control ID
Scheme'
------------------------------
RISULTATO STEP 5
------------------------------
Nuove coppie generate: 206
Codici BSI in Fabasoft non mappati in EUCS: 1

File salvato in: TrainAndTestData/trainingData/training_data_step5_fabasoft_eucs.csv
                         anchor_id target_id
0  NumberOfKnownLowVulnerabilities  OPS-17.1
1  NumberOfKnownLowVulnerabilities  OPS-17.2
2  NumberOfKnownLowVulnerabilities  OPS-17.3


### BSIC5 and Fabasoft

In [8]:
import pandas as pd
import json
import re
import os

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
PATH_FABASOFT = "Schemes/fabasoftMetrics.csv"
PATH_BSI = "Schemes/BSI-C5.json"
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step6_fabasoft_bsi.csv"

# ==============================================================================
# 2. CARICAMENTO E INDICIZZAZIONE BSI C5 (JSON)
# ==============================================================================
print("1. Caricamento e indicizzazione BSI C5...")

try:
    with open(PATH_BSI, 'r', encoding='utf-8') as f:
        bsi_data = json.load(f)
    print(f"   - BSI C5 JSON caricato. Totale elementi: {len(bsi_data)}")
except Exception as e:
    print(f"   - ERRORE CRITICO BSI: {e}")
    exit()

bsi_lookup = {}

for item in bsi_data:
    if 'code' not in item or not item['code']:
        continue
        
    # Normalizzazione ID (es. "OPS-22" -> "ops-22")
    code = str(item['code']).strip().lower()
    
    # Costruzione Testo Ricco
    name = item.get('name', '') or ""
    desc_title = item.get('descriptionTitle', '') or ""
    desc_ext = item.get('descriptionExtended', '') or ""
    
    full_text = f"{name}: {desc_title} {desc_ext}".strip()
    full_text = re.sub(r'\s+', ' ', full_text).strip()
    
    if full_text:
        bsi_lookup[code] = full_text

print(f"   - Indicizzati {len(bsi_lookup)} controlli BSI con testo.")

# ==============================================================================
# 3. ELABORAZIONE FABASOFT METRICS -> BSI DIRECT
# ==============================================================================
print("\n2. Elaborazione Fabasoft Metrics -> BSI C5 (Direct)...")

try:
    # Engine python per gestire header multi-riga e quote
    df_fab = pd.read_csv(PATH_FABASOFT, engine='python', on_bad_lines='skip')
    print(f"   - Fabasoft caricato: {len(df_fab)} righe.")
except Exception as e:
    print(f"   - ERRORE Fabasoft: {e}")
    exit()

# Trova la colonna giusta (quella che contiene "Possible Control ID")
ref_col = None
for col in df_fab.columns:
    if "Possible Control ID" in col:
        ref_col = col
        break
# Fallback
if not ref_col and not df_fab.empty:
    ref_col = df_fab.columns[-1]

print(f"   - Colonna Riferimenti Fabasoft: '{ref_col}'")

training_data = []
stats = {"matches": 0, "missing": set()}

if not df_fab.empty and ref_col:
    for idx, row in df_fab.iterrows():
        # 1. Anchor (Metrica)
        m_id = str(row.get('ID', ''))
        m_name = str(row.get('Name', ''))
        m_desc = str(row.get('Description', ''))
        
        # Pulizia stringhe "nan"
        if m_id.lower() == 'nan': m_id = ""
        if m_name.lower() == 'nan': m_name = ""
        if m_desc.lower() == 'nan': m_desc = ""
        
        anchor_parts = [p for p in [m_id, m_name, m_desc] if p]
        anchor_text = ": ".join(anchor_parts)
        anchor_text = re.sub(r'\s+', ' ', anchor_text).strip()
        
        if not anchor_text: continue
        
        # 2. Estrazione Riferimenti BSI
        raw_refs = str(row[ref_col]) if not pd.isna(row[ref_col]) else ""
        
        # Il contenuto è tipo "BSI C5 OPS-22\nBSI C5 PSS-02"
        lines = raw_refs.split('\n')
        
        for line in lines:
            line = line.strip()
            
            # Regex per estrarre il codice dopo "BSI C5"
            # Cerca "BSI C5" (case insensitive) seguito da spazi opzionali e poi il codice
            match = re.search(r'BSI\s*C5\s*([\w-]+)', line, re.IGNORECASE)
            
            if match:
                target_code = match.group(1).lower() # es. "ops-22"
                
                # 3. Matching
                if target_code in bsi_lookup:
                    positive_text = bsi_lookup[target_code]
                    
                    training_data.append({
                        'anchor_id': m_id,
                        'anchor_text': anchor_text,
                        'target_id': target_code,
                        'target_text': positive_text,
                        'source': 'Fabasoft Metrics',
                        'target': 'BSI C5'
                    })
                    stats["matches"] += 1
                else:
                    stats["missing"].add(target_code)

# ==============================================================================
# 4. SALVATAGGIO
# ==============================================================================
df_result = pd.DataFrame(training_data)
df_result.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO STEP 6")
print("-" * 30)
print(f"Nuove coppie generate: {len(df_result)}")
if stats['missing']:
    print(f"Codici BSI mancanti: {len(stats['missing'])} (es: {list(stats['missing'])[:3]})")
print(f"\nFile salvato in: {OUTPUT_FILE}")

if not df_result.empty:
    print(df_result[['anchor_id', 'target_id']].head(3))

1. Caricamento e indicizzazione BSI C5...
   - BSI C5 JSON caricato. Totale elementi: 294
   - Indicizzati 223 controlli BSI con testo.

2. Elaborazione Fabasoft Metrics -> BSI C5 (Direct)...
   - Fabasoft caricato: 57 righe.
   - Colonna Riferimenti Fabasoft: 'Possible Control ID
Scheme'
------------------------------
RISULTATO STEP 6
------------------------------
Nuove coppie generate: 27
Codici BSI mancanti: 1 (es: ['idm-11'])

File salvato in: TrainAndTestData/trainingData/training_data_step6_fabasoft_bsi.csv
                            anchor_id target_id
0     NumberOfKnownLowVulnerabilities    ops-22
1     NumberOfKnownLowVulnerabilities    pss-02
2  NumberOfKnownMediumVulnerabilities    ops-22


### newEUCS and BSIC5 no old data overlap from oldEUCS

In [9]:
import pandas as pd
import json
import re
import os

# ==============================================================================
# 1. CONFIGURAZIONE PERCORSI
# ==============================================================================
PATH_NEW_EUCS = "Schemes/NewEucsRequirements_with_texts.csv"
PATH_OLD_EUCS = "Schemes/OldEucsRequirements.csv"
PATH_BSI = "Schemes/BSI-C5.json"
OUTPUT_FILE = "TrainAndTestData/trainingData/training_data_step7_neweucs_bsi_filtered.csv"

# ==============================================================================
# 2. CARICAMENTO "OLD EUCS" (DA ESCLUDERE)
# ==============================================================================
print("1. Caricamento Old EUCS (Blacklist)...")
old_eucs_codes = set()

try:
    # Provo a leggere il vecchio file
    df_old = pd.read_csv(PATH_OLD_EUCS)
    
    # Identifico la colonna ID (dal tuo head sembra 'controlId')
    col_id_old = None
    for c in df_old.columns:
        if 'controlId' in c or 'ID' in c:
            col_id_old = c
            break
            
    if col_id_old:
        # Normalizzo e salvo nel set
        old_eucs_codes = set(df_old[col_id_old].dropna().astype(str).str.strip().str.lower())
        print(f"   - Trovati {len(old_eucs_codes)} codici 'Old EUCS' da escludere dal training.")
    else:
        print("   - ATTENZIONE: Colonna ID non trovata in Old EUCS. Nessun filtro applicato!")

except Exception as e:
    print(f"   - Errore caricamento Old EUCS: {e}")
    print("   - Procedo senza filtrare (rischio data leakage se questo file serviva per il test).")

# ==============================================================================
# 3. INDICIZZAZIONE BSI C5 (TARGET)
# ==============================================================================
print("\n2. Indicizzazione BSI C5...")
bsi_lookup = {}

try:
    with open(PATH_BSI, 'r', encoding='utf-8') as f:
        bsi_data = json.load(f)
        
    for item in bsi_data:
        if 'code' not in item: continue
        
        # ID BSI normalizzato
        code = str(item['code']).strip().lower()
        
        # Testo Ricco
        name = item.get('name', '') or ""
        desc_t = item.get('descriptionTitle', '') or ""
        desc_e = item.get('descriptionExtended', '') or ""
        
        full_text = f"{name}: {desc_t} {desc_e}".strip()
        full_text = re.sub(r'\s+', ' ', full_text).strip()
        
        if full_text:
            bsi_lookup[code] = full_text

    print(f"   - Indicizzati {len(bsi_lookup)} controlli BSI.")

except Exception as e:
    print(f"   - ERRORE CRITICO BSI: {e}")
    exit()

# ==============================================================================
# 4. MAPPING NEW_EUCS -> BSI (CON FILTRO)
# ==============================================================================
print("\n3. Generazione coppie NewEUCS -> BSI (Filtrate)...")

training_data = []
stats = {
    "total_candidates": 0,
    "excluded_by_filter": 0,
    "matches": 0,
    "missing_bsi": 0
}

try:
    # Caricamento New EUCS con pulizia colonne
    df_new = pd.read_csv(PATH_NEW_EUCS, sep=',', engine='python', on_bad_lines='skip')
    # Se fallisce virgola, prova ;
    if 'C5.2020 GERMANY' not in df_new.columns and len(df_new.columns) < 5:
         df_new = pd.read_csv(PATH_NEW_EUCS, sep=';', engine='python', on_bad_lines='skip')
         
    df_new.columns = df_new.columns.str.strip() # Rimuove spazi dai nomi colonne

    # Colonne chiave
    col_new_id = 'EUCS Ref (Detailed)' # ID univoco del controllo EUCS
    col_new_text = 'EUCS Text'
    col_bsi_ref = 'C5.2020 GERMANY'

    # Verifica colonne
    if col_new_id not in df_new.columns or col_bsi_ref not in df_new.columns:
        print(f"   - Colonne mancanti in New EUCS. Trovate: {df_new.columns.tolist()}")
        exit()

    for idx, row in df_new.iterrows():
        # 1. Analisi Anchor (EUCS)
        eucs_id_raw = str(row[col_new_id])
        eucs_id_clean = eucs_id_raw.strip().lower()
        
        eucs_text = str(row[col_new_text]) if col_new_text in row else ""
        eucs_text = re.sub(r'\s+', ' ', eucs_text).strip()

        if pd.isna(row[col_new_id]) or not eucs_text:
            continue

        # 2. FILTRO ANTI-LEAKAGE
        # Se questo ID EUCS è presente nel vecchio set (OldEucs), SALTALO.
        if eucs_id_clean in old_eucs_codes:
            stats["excluded_by_filter"] += 1
            continue

        # 3. Analisi Target (BSI)
        bsi_refs_raw = str(row[col_bsi_ref])
        if pd.isna(row[col_bsi_ref]) or not bsi_refs_raw.strip():
            continue

        # Split refs (newline o virgola)
        refs = re.split(r'[\n,]', bsi_refs_raw)
        
        for r in refs:
            stats["total_candidates"] += 1
            target_bsi = r.strip().lower()
            
            if not target_bsi: continue

            if target_bsi in bsi_lookup:
                # MATCH!
                target_text = bsi_lookup[target_bsi]
                
                training_data.append({
                    'anchor_id': eucs_id_raw, # Mantengo ID originale per leggibilità
                    'anchor_text': eucs_text,
                    'target_id': target_bsi,
                    'target_text': target_text,
                    'source': 'NewEUCS (Filtered)',
                    'target': 'BSI C5'
                })
                stats["matches"] += 1
            else:
                stats["missing_bsi"] += 1

except Exception as e:
    print(f"   - ERRORE durante il processing: {e}")

# ==============================================================================
# 5. SALVATAGGIO
# ==============================================================================
df_result = pd.DataFrame(training_data)
df_result.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("RISULTATO STEP 7 (FILTRATO)")
print("-" * 30)
print(f"Controlli Old EUCS trovati (Blacklist): {len(old_eucs_codes)}")
print(f"Coppie potenziali scartate perché nel Test Set: {stats['excluded_by_filter']}")
print(f"Nuove coppie valide generate: {len(df_result)}")
print(f"File salvato in: {OUTPUT_FILE}")

if not df_result.empty:
    print("\nAnteprima:")
    print(df_result[['anchor_id', 'target_id']].head(3))

1. Caricamento Old EUCS (Blacklist)...
   - Trovati 70 codici 'Old EUCS' da escludere dal training.

2. Indicizzazione BSI C5...
   - Indicizzati 223 controlli BSI.

3. Generazione coppie NewEUCS -> BSI (Filtrate)...
------------------------------
RISULTATO STEP 7 (FILTRATO)
------------------------------
Controlli Old EUCS trovati (Blacklist): 70
Coppie potenziali scartate perché nel Test Set: 0
Nuove coppie valide generate: 569
File salvato in: TrainAndTestData/trainingData/training_data_step7_neweucs_bsi_filtered.csv

Anteprima:
  anchor_id target_id
0  OIS-01.1    ois-01
1  OIS-01.2    ois-01
2  OIS-01.3    ois-01


## Merge to creare train data

In [10]:
import pandas as pd
import os

# ==============================================================================
# 1. CONFIGURAZIONE
# ==============================================================================
DATA_DIR = "TrainAndTestData/trainingData/"
OUTPUT_MASTER = "TrainAndTestData/trainingData/MASTER_TRAINING_DATA.csv"

# Lista completa dei 7 file (Tutti gli step fatti finora)
FILES_TO_MERGE = [
    "training_data_step1_cisco_ens.csv",            # Cisco -> Spanish ENS
    "training_data_step2_cisco_secNumCloud.csv",    # Cisco -> SecNumCloud
    "training_data_step3_cisco_bsi.csv",            # Cisco -> BSI C5
    "training_data_step4_cisco_eucs.csv",           # Cisco -> EUCS (Originale)
    "training_data_step5_fabasoft_eucs.csv",        # Fabasoft -> EUCS
    "training_data_step6_fabasoft_bsi.csv",         # Fabasoft -> BSI C5
    "training_data_step7_neweucs_bsi_filtered.csv"  # NewEUCS -> BSI (Filtrato Old)
]

# ==============================================================================
# 2. UNIONE (MERGE)
# ==============================================================================
print("Inizio creazione del MASTER DATASET COMPLETO...")

df_list = []

for filename in FILES_TO_MERGE:
    file_path = os.path.join(DATA_DIR, filename)
    
    if os.path.exists(file_path):
        print(f"   - Lettura: {filename} ...", end="")
        try:
            df = pd.read_csv(file_path)
            # Aggiungiamo colonna origine per tracciabilità
            df['original_file'] = filename
            df_list.append(df)
            print(f" OK ({len(df)} righe)")
        except Exception as e:
            print(f" ERRORE: {e}")
    else:
        print(f"   - ATTENZIONE: File mancante: {filename}")

if not df_list:
    print("ERRORE: Nessun file trovato!")
    exit()

# Concatenazione
master_df = pd.concat(df_list, ignore_index=True)

# ==============================================================================
# 3. SHUFFLE E PULIZIA
# ==============================================================================
print("\nMescolamento e Pulizia Finale...")

# 1. Shuffle (Mescolamento casuale)
# shuffle al 100% per rompere l'ordine dei file
master_df = master_df.sample(frac=1, random_state=42).reset_index(drop=True)

# 2. Rimozione duplicati esatti
# Se una coppia è identica (stessa ancora, stesso target), la teniamo una volta sola
initial_len = len(master_df)
master_df.drop_duplicates(subset=['anchor_text', 'target_text'], inplace=True)
dedup_len = len(master_df)

if initial_len != dedup_len:
    print(f"   - Rimossi {initial_len - dedup_len} duplicati esatti.")

# 3. Pulizia NaN
master_df.dropna(subset=['anchor_text', 'target_text'], inplace=True)

# ==============================================================================
# 4. SALVATAGGIO
# ==============================================================================
master_df.to_csv(OUTPUT_MASTER, index=False, encoding='utf-8')

print("-" * 30)
print("RISULTATO FINALE (MASTER DATASET)")
print("-" * 30)
print(f"Totale righe nel dataset di training: {len(master_df)}")
print("\nDistribuzione per Schema Target:")
# Mostra quante coppie abbiamo per ogni tipo di target
print(master_df['target'].value_counts())

print(f"\nFile MASTER salvato in: {OUTPUT_MASTER}")
print("\nAnteprima finale:")
print(master_df[['source', 'target', 'anchor_id', 'target_id']].head(5))

Inizio creazione del MASTER DATASET COMPLETO...
   - Lettura: training_data_step1_cisco_ens.csv ... OK (318 righe)
   - Lettura: training_data_step2_cisco_secNumCloud.csv ... OK (567 righe)
   - Lettura: training_data_step3_cisco_bsi.csv ... OK (555 righe)
   - Lettura: training_data_step4_cisco_eucs.csv ... OK (1272 righe)
   - Lettura: training_data_step5_fabasoft_eucs.csv ... OK (206 righe)
   - Lettura: training_data_step6_fabasoft_bsi.csv ... OK (27 righe)
   - Lettura: training_data_step7_neweucs_bsi_filtered.csv ... OK (569 righe)

Mescolamento e Pulizia Finale...
   - Rimossi 6 duplicati esatti.
------------------------------
RISULTATO FINALE (MASTER DATASET)
------------------------------
Totale righe nel dataset di training: 3508

Distribuzione per Schema Target:
target
EUCS                 1270
BSI C5               1151
SecNumCloud (EN)      563
Spanish ENS (EN)      318
EUCS (via BSI C5)     206
Name: count, dtype: int64

File MASTER salvato in: TrainAndTestData/trainingDat

## Generate the 2 test data

one is the old test data from the old paper (oldEucs and medinaMetrics) and the other (the one that is made by old data but it was not use for testing) is oldEucs and bsic5